In [1]:
// Making REST call using scalaj-http library
//JSON parsing of REST results using play-json library
//Other important Imports
%AddJar -magic http://central.maven.org/maven2/org/scalaj/scalaj-http_2.10/2.3.0/scalaj-http_2.10-2.3.0.jar
%AddJar -magic http://central.maven.org/maven2/com/typesafe/play/play-json_2.10/2.4.6/play-json_2.10-2.4.6.jar
import scalaj.http._
import play.api.libs.json._
import scala.util.{Try,Success,Failure}

Using cached version of scalaj-http_2.10-2.3.0.jar
Using cached version of play-json_2.10-2.4.6.jar


In [2]:
// loading test dataset from Udacity Open Source Sample Data 
val UdacityDataURL = "https://github.com/smatlapudi/testdrive-bluemix-driverbehavior-service/raw/master/udacity-ch3-north/testdata/udacity-ch3-north.json"
val MastikshTestData =  scala.io.Source.fromURL(UdacityDataURL).getLines.to[collection.immutable.Seq]
//Create SCALA RDD from Udacity sample data 
val tripDataLines_rdd = sc.parallelize(MastikshTestData)
tripDataLines_rdd.coalesce(1).saveAsTextFile("MastikshTestData1")
//check sample data to make sure RDD loaded properly
tripDataLines_rdd.take(5)

Array({"timestamp":"2016-11-17T22:23:03Z","heading":357.4195341311482,"speed":15.085078615161539,"latitude":37.4019584656,"longitude":-122.114883423}, {"timestamp":"2016-11-17T22:23:04Z","heading":357.12198437345154,"speed":15.001162178540001,"latitude":37.4020271301,"longitude":-122.115020752}, {"timestamp":"2016-11-17T22:23:05Z","heading":355.95965463973073,"speed":14.618271412569998,"latitude":37.4021072388,"longitude":-122.115158081}, {"timestamp":"2016-11-17T22:23:06Z","heading":330.98606173371456,"speed":14.17331637527667,"latitude":37.4021835327,"longitude":-122.115287781}, {"timestamp":"2016-11-17T22:23:07Z","heading":351.05080395396425,"speed":13.715587838680003,"latitude":37.4022636414,"longitude":-122.115409851})

In [42]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.SparkSession  
import org.apache.spark.sql.types.{StructType, StructField, StringType, IntegerType}
val session = SparkSession.builder().appName("Mastiksh").master("local").getOrCreate()
case class TripDataPoint(timestamp: String, heading: Double, speed: Double, latitude: Double, longitude: Double )
val jsonSchema = new StructType().add("timestamp", "string").add("heading", "double").add("speed", "double").add("latitude", "double").add("longitude", "double")
// Convert tripDataLines_rdd to TripRdd 
val streamingInputDF = session.read.schema(jsonSchema).json(tripDataLines_rdd)
streamingInputDF.show(5)

+--------------------+------------------+------------------+-------------+--------------+
|           timestamp|           heading|             speed|     latitude|     longitude|
+--------------------+------------------+------------------+-------------+--------------+
|2016-11-17T22:23:03Z| 357.4195341311482|15.085078615161539|37.4019584656|-122.114883423|
|2016-11-17T22:23:04Z|357.12198437345154|15.001162178540001|37.4020271301|-122.115020752|
|2016-11-17T22:23:05Z|355.95965463973073|14.618271412569998|37.4021072388|-122.115158081|
|2016-11-17T22:23:06Z|330.98606173371456| 14.17331637527667|37.4021835327|-122.115287781|
|2016-11-17T22:23:07Z|351.05080395396425|13.715587838680003|37.4022636414|-122.115409851|
+--------------------+------------------+------------------+-------------+--------------+
only showing top 5 rows



In [43]:
//Check Schema
streamingInputDF.printSchema()

root
 |-- timestamp: string (nullable = true)
 |-- heading: double (nullable = true)
 |-- speed: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)



In [29]:
//Register the DataFrame as a table so that you can run SQL queries on it and show the first two rows:
streamingInputDF.registerTempTable("Mastiksh_Telematics")
val Mastiksh_Table1 = session.sql("SELECT * FROM Mastiksh_Telematics limit")
Mastiksh_Table1.show(5)
Mastiksh_Table1.count()

+--------------------+------------------+------------------+-------------+--------------+
|           timestamp|           heading|             speed|     latitude|     longitude|
+--------------------+------------------+------------------+-------------+--------------+
|2016-11-17T22:23:03Z| 357.4195341311482|15.085078615161539|37.4019584656|-122.114883423|
|2016-11-17T22:23:04Z|357.12198437345154|15.001162178540001|37.4020271301|-122.115020752|
|2016-11-17T22:23:05Z|355.95965463973073|14.618271412569998|37.4021072388|-122.115158081|
|2016-11-17T22:23:06Z|330.98606173371456| 14.17331637527667|37.4021835327|-122.115287781|
|2016-11-17T22:23:07Z|351.05080395396425|13.715587838680003|37.4022636414|-122.115409851|
+--------------------+------------------+------------------+-------------+--------------+
only showing top 5 rows



2767

In [47]:
// loading test dataset from Udacity Open Source Sample Data 
val UdacityDRURL = "https://raw.githubusercontent.com/smatlapudi/testdrive-bluemix-driverbehavior-service/master/udacity-ch3-north/result/udacity-ch3-north-driverBehaviorResult.json"
val MastikshTestDRData =  scala.io.Source.fromURL(UdacityDRURL).getLines.to[collection.immutable.Seq]
//Create SCALA RDD from Udacity sample data 
val tripDRLines_rdd = sc.parallelize(MastikshTestDRData)
tripDRLines_rdd.coalesce(1).saveAsTextFile("MastikshTestDRData3")
//check sample data to make sure RDD loaded properly
tripDRLines_rdd.take(20)

Array({, "  "id" : {", "    "trip_uuid" : "7fbdc6af-179b-4346-9f2d-65d382f30429",", "    "tenant_id" : "c97244f8-e5b1-4982-94c4-0b4bd5c5b711"", "  },", "  "end_altitude" : 0.0,", "  "end_latitude" : 37.5503832582108,", "  "end_longitude" : -122.3120112947789,", "  "end_time" : 1479424149000,", "  "generated_time" : 1480987518794,", "  "mo_id" : "udacity-ch3",", "  "driver_id" : "",", "  "start_altitude" : 0.0,", "  "start_latitude" : 37.4019766,", "  "start_longitude" : -122.114744,", "  "start_time" : 1479421383000,", "  "trip_id" : "udacity-ch3-north",", "  "ctx_sub_trips" : [ {", "    "id" : {", "      "sub_trip_id" : "d4f784c9-21b1-46cd-8560-e218b1e3853b",")

In [32]:
//Create an Apache® Spark machine learning model
//Prepare data : split your data into: train, test and predict data sets
val splits = streamingInputDF.randomSplit(Array(0.8, 0.18, 0.02), seed = 24L)
val Mks_training_data = splits(0).cache()
val Mks_test_data = splits(1)
val Mks_prediction_data = splits(2)

println("Number of training records: ", Mks_training_data.count())
println("Number of testing records : ", Mks_test_data.count())
println("Number of prediction records : ", Mks_prediction_data.count())

(Number of training records: ,2236)
(Number of testing records : ,482)
(Number of prediction records : ,49)


In [39]:
// Spark ML libraries
import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.feature.{OneHotEncoder, StringIndexer, IndexToString, VectorAssembler}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.{Model, Pipeline, PipelineStage, PipelineModel}
import org.apache.spark.sql.SparkSession